In [ ]:
# !pip3 install matplotlib
# !pip3 install seaborn
# !pip3 install -U scikit-learn
# !pip3 install tensorflow
# !pip3 install --upgrade tensorflow

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
# from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

LOAD DATASET : KAGGLE

In [ ]:
# data_dir = 'C:/Users/aditi/OneDrive/Desktop/DESKTOP/VS_LANG/Python/LEGALITY_ML/DATASETS/BHSig260-Bengali/BHSig260-Bengali'
data_dir = 'C:/Users/aditi/OneDrive/Desktop/DESKTOP/VS_LANG/Python/LEGALITY_ML/DATASETS/archive/sign_data/sign_data'
# data_dir = 'C:/Users/aditi/OneDrive/Desktop/DESKTOP/VS_LANG/Python/LEGALITY_ML/DATASETS/data/data'

img_width, img_height = 256, 256

datagen = ImageDataGenerator(rescale=1./255,
                             rotation_range=40,
                             width_shift_range=0.4,
                             height_shift_range=0.4,
                             shear_range=0.4,
                             zoom_range=0.4,
                             horizontal_flip=True,
                             vertical_flip=True,
                             validation_split=0.2)

train_data = datagen.flow_from_directory(data_dir,
                                         target_size=(img_width, img_height),
                                         batch_size=64,
                                         class_mode='binary',
                                         subset='training')

validation_data = datagen.flow_from_directory(data_dir,
                                            target_size=(img_width, img_height),
                                            batch_size=64,
                                            class_mode='binary',
                                            subset='validation')

In [ ]:
# class_weights = {0: 329/429, 1: 100/429}

CNN ARCHITECTURE

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(1024, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

CNN MODEL COMPILATION

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
            #   optimizer='sdp',
            #   optimizer='rmsprop',
              metrics=['accuracy', 'precision', 'recall'])

CNN MODEL TRAINING

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, mode='min', min_lr=0.001)


history = model.fit(train_data,
                    epochs=100,
                    validation_data=validation_data,
                    callbacks=[es, rlr],
                    # class_weight=class_weights
                    )

CNN MODEL EVALUATION

In [ ]:
y_pred = model.predict(validation_data)
y_pred = (y_pred > 0.5)

In [ ]:
scores = model.evaluate(validation_data, verbose=1)
f1 = f1_score(validation_data.classes, y_pred)
print('Accuracy:', scores[1])
print('Precision:', scores[2])
print('Recall:', scores[3])  
print('F1 Score:', f1)                       

CLASSIFICATION REPORT & CONFUSION MATRIX - CNN

In [ ]:
cm = confusion_matrix(validation_data.classes, y_pred)
cr = classification_report(validation_data.classes, y_pred)

In [ ]:
print('CLASSIFICATION REPORT: \n',cr)
print('CONFUSION MATRIX: \n', cm)

TRAINING HISTORY PLOT

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()